In [43]:
import datetime as dt
import pandas as pd

from fmiopendata.wfs import download_stored_query

Vähennetään end_time ajasta 3 tuntia, koska datan viemisessä internettiin kestää hetken ja näin vältytään tyhjiltä (NaN) arvoilta

In [44]:
# Retrieve the latest hour of data from a bounding box
end_time = dt.datetime.now() - dt.timedelta(hours=3)
start_time = end_time - dt.timedelta(days=7)
# Convert times to properly formatted strings
start_time = start_time.isoformat(timespec="seconds") + "Z"
# -> 2025-09-28T12:00:00Z
end_time = end_time.isoformat(timespec="seconds") + "Z"
# -> 2025-09-28T13:00:00Z

Katsotaan minkälaisia mittauspisteitä määrittelemälläni alueelta löytyy. Tämän perusteella valitaan Tampereen ja Helsingin keskustojen läheisyydessä sijaitsevat mittauspisteet.

In [45]:
obs = download_stored_query("urban::observations::airquality::hourly::multipointcoverage", # ILMANLAATUPARAMETRI
                            args=["bbox=23,60,26,63",
                                  "starttime=" + start_time,
                                  "endtime=" + end_time])

df

,datetime,location,Sulphur dioxide,Nitrogen monoxide,Nitrogen dioxide,Ozone,Odorous sulphur compounds,Carbon monoxide,Particulate matter < 10 µm,Particulate matter < 2.5 µm,Air Quality Index,musta hiili PM2.5
0,2025-09-26 14:00:00,Helsinki Kallio 2,0.7,0.5,4.6,56.4,NaN,NaN,4.8,1.6,1.0,0.11
1,2025-09-26 14:00:00,Espoo Leppävaara Läkkisepänkuja,NaN,1.4,5.3,NaN,NaN,NaN,5.1,1.6,1.0,0.20
2,2025-09-26 14:00:00,Espoo Luukki,-0.4,-0.1,0.8,56.2,NaN,NaN,3.9,1.3,1.0,0.05
3,2025-09-26 14:00:00,Helsinki Mannerheimintie,NaN,10.7,16.2,NaN,NaN,NaN,10.4,3.1,1.0,0.30
4,2025-09-26 14:00:00,Vantaa Tikkurila Neilikkatie,NaN,0.9,5.1,NaN,NaN,NaN,5.5,1.6,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1387,2025-09-28 13:00:00,Hollola Kuntotie,NaN,NaN,2.2,NaN,NaN,NaN,5.1,2.0,1.0,NaN
1388,2025-09-28 13:00:00,Helsinki Teollisuuskatu,NaN,1.2,4.0,54.8,NaN,NaN,4.1,1.3,1.0,0.10
1389,2025-09-28 13:00:00,Helsinki Tapanila 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.09
1390,2025-09-28 13:00:00,Vantaa Kehä III Viinikkala,NaN,2.0,8.0,NaN,NaN,NaN,4.0,1.6,1.0,NaN


Tarkistetaan, että datan tyypitykset ovat oikein jatkokäsittelyä varten.
Tässä esimerkissä datatyypit ovat datetime64[ns], object ja float64.

In [46]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1392 entries, 0 to 1391
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   datetime                     1392 non-null   datetime64[ns]
 1   location                     1392 non-null   object        
 2   Sulphur dioxide              288 non-null    float64       
 3   Nitrogen monoxide            624 non-null    float64       
 4   Nitrogen dioxide             1248 non-null   float64       
 5   Ozone                        384 non-null    float64       
 6   Odorous sulphur compounds    48 non-null     float64       
 7   Carbon monoxide              0 non-null      float64       
 8   Particulate matter < 10 µm   1152 non-null   float64       
 9   Particulate matter < 2.5 µm  1152 non-null   float64       
 10  Air Quality Index            1344 non-null   float64       
 11  musta hiili PM2.5            336 non-null  

Valitaan mittauspisteiksi: 
* Tampere Linja-autoasema
* Helsinki Mannerheimintie

Valitaan mittausdatasta tarkasteltavaksi:
* datetime
* location
* Particular matter < 2.5 µm
* Air Quality Index
* Nitrogen dioxide

In [47]:
records = []

for datet, locations in obs.data.items():
    for location, pollutants in locations.items():
        record = {'datetime': datet, 'location': location}
        for pollutant, values in pollutants.items():
            record[pollutant] = values['value']
        records.append(record)

df = pd.DataFrame(records)

filtered_df = df[df['location'].isin(['Tampere Linja-autoasema','Helsinki Mannerheimintie'])][['datetime','location', 'Particulate matter < 2.5 µm','Air Quality Index', 'Nitrogen dioxide']]

filtered_df

,datetime,location,Particulate matter < 2.5 µm,Air Quality Index,Nitrogen dioxide
3,2025-09-21 16:00:00,Helsinki Mannerheimintie,9.3,1.0,18.6
9,2025-09-21 16:00:00,Tampere Linja-autoasema,5.2,1.0,11.6
32,2025-09-21 17:00:00,Helsinki Mannerheimintie,9.9,1.0,11.6
38,2025-09-21 17:00:00,Tampere Linja-autoasema,4.5,1.0,7.4
61,2025-09-21 18:00:00,Helsinki Mannerheimintie,10.4,2.0,6.4
...,...,...,...,...,...
4794,2025-09-28 13:00:00,Tampere Linja-autoasema,0.3,1.0,5.9
4817,2025-09-28 14:00:00,Helsinki Mannerheimintie,3.4,1.0,8.2
4823,2025-09-28 14:00:00,Tampere Linja-autoasema,1.6,1.0,8.3
4846,2025-09-28 15:00:00,Helsinki Mannerheimintie,3.4,1.0,8.6


Tallenetaan datataulu CSV-muotoon, jota voimme käyttää pohjana Power BI -raportille. Tallenetaan datataulu suoraan /data/processed/ -kansioon.

In [48]:
filtered_df.to_csv("/users/raunopaukkeri/Kaisuhamk/bi_project/data/processed/ilmanlaatu_tampere_helsinki.csv", index=False)

In [49]:
filtered_df.head()

,datetime,location,Particulate matter < 2.5 µm,Air Quality Index,Nitrogen dioxide
3,2025-09-21 16:00:00,Helsinki Mannerheimintie,9.3,1.0,18.6
9,2025-09-21 16:00:00,Tampere Linja-autoasema,5.2,1.0,11.6
32,2025-09-21 17:00:00,Helsinki Mannerheimintie,9.9,1.0,11.6
38,2025-09-21 17:00:00,Tampere Linja-autoasema,4.5,1.0,7.4
61,2025-09-21 18:00:00,Helsinki Mannerheimintie,10.4,2.0,6.4
